In [42]:
import os
import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx
import matplotlib.pylab as plt

%matplotlib inline

from urllib.request import urlopen
from pathlib import Path

## Carga de datos

In [2]:
red_lit = nx.read_edgelist('./Material_de_interes/tc02Data/yeast_LIT.txt')
red_apms = nx.read_edgelist('./Material_de_interes/tc02Data/yeast_AP-MS.txt')
red_bin = nx.read_edgelist('./Material_de_interes/tc02Data/yeast_Y2H.txt')

In [44]:
escenciales = open('./Datos/Essential_ORFs_paperHe2.txt')
lista_escenciales = []
for line in escenciales:
    lista_escenciales.append(line.rstrip('\n'))

In [59]:
def nodos_esenciales(red, lista_atr = lista_escenciales, atributo = 'Escencial'):
    
    dict_vacio = dict()
    nx.set_node_attributes(red, dict_vacio.fromkeys(list(red.nodes()),0), atributo )
    
    for nodo in lista_atr:
        try:
            red.nodes[nodo][atributo] = 1
        except KeyError:
            continue
            
    return None

In [64]:
nodos_esenciales(red_lit)
nodos_esenciales(red_bin)
nodos_esenciales(red_apms)

In [3]:
red_df = pd.read_csv('./Material_de_interes/tc02Data/yeast_LIT_Reguly.txt',sep="\t", header=0)

In [4]:
red_df.head()

,Bait gene/protein,Hit gene/protein,Bait Standard Name,Hit Standard Name,Experimental System,Source,PubMed ID
0,YBR236C,YML010W,ABD1,SPT5,Affinity Capture-MS,Lindstrom DL et al,12556496
1,YML010W,YBR236C,SPT5,ABD1,Affinity Capture-MS,Lindstrom DL et al,12556496
2,YDR448W,YPL254W,ADA2,HFI1,Affinity Capture-MS,Rodriguez-Navarro S et al,14718168
3,YDR448W,YDR176W,ADA2,NGG1,Affinity Capture-MS,Rodriguez-Navarro S et al,14718168
4,YDR448W,YOR257W,ADA2,CDC31,Affinity Capture-MS,Rodriguez-Navarro S et al,14718168


## Características de las redes

#### Tabla 1

In [5]:
import herramientas_redes as hr

In [6]:
redes = [red_apms, red_bin, red_lit]
props = []

for red in redes:
    props.append(hr.propiedades(red))
sRed = ['Red AP-MS','Red Y2H', 'Red Literatura']

In [7]:
pd.DataFrame(props, index=sRed).drop(['Densidad', 'Diámetro', 'Grado max', 'Grado min', 'Transitividad'], axis = 1)

,Clustering medio,Enlaces,Grado promedio,Nodos
Red AP-MS,0.554636,9070,11.183724,1622
Red Y2H,0.046194,2930,2.903865,2018
Red Literatura,0.292492,2925,3.808594,1536


#### Tabla 2

In [8]:
def overlap_enlaces(red1, red2):
    
    nodos = set(list(red1.nodes())+list(red2.nodes()))
    counter = 0
    for nodo in nodos:
        try:
            l1 = list(red1[nodo])
            l2 = list(red2[nodo])
            if l1 and l2:
                counter += len(hr.intersect(l1,l2))
        except KeyError:
            continue
    cantidad = counter/2
    
    return [ cantidad/ red1.number_of_edges(), cantidad/ red2.number_of_edges()]

In [9]:
lista_apms_lit = overlap_enlaces(red_apms, red_lit)
lista_lit_y2h = overlap_enlaces(red_lit,red_bin)
lista_y2h_apms = overlap_enlaces(red_bin,red_apms)

In [10]:
tabla = pd.DataFrame(np.nan, columns = sRed, index = sRed)
tabla.loc['Red AP-MS']['Red Y2H'] = lista_y2h_apms[1]
tabla.loc['Red Y2H']['Red AP-MS'] = lista_y2h_apms[0]
tabla.loc['Red AP-MS']['Red Literatura'] = lista_apms_lit[0]
tabla.loc['Red Literatura']['Red AP-MS'] = lista_apms_lit[1]
tabla.loc['Red Literatura']['Red Y2H'] = lista_lit_y2h[0]
tabla.loc['Red Y2H']['Red Literatura'] = lista_lit_y2h[1]
tabla.loc['Red Y2H']['Red Y2H'] = 1 #'Red Y2H'
tabla.loc['Red Literatura']['Red Literatura'] = 1 #'Red Literatura'
tabla.loc['Red AP-MS']['Red AP-MS'] = 1 #'Red AP-MS'

In [11]:
tabla

,Red AP-MS,Red Y2H,Red Literatura
Red AP-MS,1.000000,0.028666,0.143109
Red Y2H,0.088737,1.000000,0.087201
Red Literatura,0.443761,0.087350,1.000000
